In [0]:
import json, os, shutil

# Requires utils.ipynb

# Consider copying hal20:/workspace/earthtime/projects/timemachine-creator

In [0]:
def prepend_to_PATH(dir):
    if dir in os.environ['PATH'].split(':'):
        print('%s already in PATH' % dir)
    else:
        print('Adding %s to PATH' % dir)
        os.environ['PATH'] = dir + ':' + os.environ['PATH']

def set_ffmpeg_path_for_timemachine(path_to_timemachine_creator):
    path_to_ffmpeg = path_to_timemachine_creator + '/tilestacktool/ffmpeg/linux/ffmpeg'
    assert(os.path.exists(path_to_ffmpeg))
    prepend_to_PATH(os.path.dirname(path_to_ffmpeg))

# quality: 24-26 are reasonable?
# capture_times is an array of strings in the format of 'YYYY' or (we think but 
# aren't yet sure) 'YYYY-MM' corresponding to the times of each frame in srcs
def create_earthtime_timemachine(timemachine, srcs, capture_times, path_to_timemachine_creator,
                                 fps=12, quality=24, overwrite=False): 
    path_to_ct_rb = path_to_timemachine_creator + '/ct/ct.rb'
    set_ffmpeg_path_for_timemachine(path_to_timemachine_creator)
    assert(os.path.exists(path_to_ct_rb))
    assert os.path.splitext(timemachine)[1] == '.timemachine'
    name = os.path.splitext(os.path.basename(timemachine))[0]

    definition = {
      "id": name,
      "label": name,
      "source": {
          "type": "images"
      },
      "videosets": [
        {
          "type": "h.264",
          "label": "600p",
          "size": [1068, 600],
          "quality": quality,
          "fps": fps
        }
      ]
    }

    tmc = os.path.splitext(timemachine)[0] + '.tmc'

    print('Creating timemachine from %d images and writing to %s with temporary directory %s' % (len(srcs), timemachine, tmc))

    if os.path.exists(tmc):
        if overwrite:
            shutil.rmtree(tmc)
        else:
            raise Exception('%s already exists, and overwrite not specified' % tmc)
    os.makedirs(tmc)
    definition_filename = tmc + '/definition.tmc'
    json.dump(definition, open(definition_filename, 'w'))
    print('Created %s definition file' % definition_filename)

    image_dir = tmc + '/0100-original-images'
    os.mkdir(image_dir)
    for src in srcs:
        os.symlink(os.path.abspath(src), image_dir + '/' + os.path.basename(src))

    ct = os.path.expanduser('~/projects/timemachine-creator/ct/ct.rb')
    cmd = [ct, '--skip-leader', '-j', '16', '-tile-mode', 'webgl', tmc, timemachine]
    print('Running %s' % (' '.join(cmd)))
    print subprocess_check(cmd)

    # Check for tm.json.  If it exists modify it to set capture-times to capture_times
    tm_filename = os.path.join(timemachine, 'tm.json')
    if not os.path.exists(tm_filename):
        print "WARNING: Can't modify tm.json.  File not found %s"%(tm_filename)
        return

    tm_json = json.load(open(tm_filename))
    tm_json['capture-times']=capture_times
    json.dump(tm_json, open(tm_filename, 'w'),indent=2)
    print "Updated capture-times in %s" % (tm_filename)

